## Train
```
python train_mlp.py --lang='de' --label_type1='pron' --label_type2='articulation' --dir_data='data/audio' --dir_list='data_prepare' --mlp_hidden=64 --epochs=200 --patience=20 --batch_size=256 --dir_model='model'
```
--lang : 언어 선택 (de-독일어, en-영어, es-스페인어, fr-프랑스어, jp-일어, ru-러시아어, zh-중국어)

--label_type1 : 학습할 발음평가 라벨 선택(1) ('pron' / 'fluency')

--label_type2 : 학습할 발음평가 라벨 선택(2) ('articulation' / 'prosody')

--dir_data : 학습할 발음평가 데이터가 위치한 폴더 경로

--dir_list : 과정 3에서 준비한 trn/val/test list가 있는 폴더 경로

--dir_model : inference를 수행할 모델 경로 

--mlp_hidden : 모델의 hidden units 개수

--epochs : finetune 훈련 횟수

--patience : 성능이 더이상 오르지 않으면 학습을 조기 종료하기 위한 파라미터

--batch_size : 훈련 batch 크기


In [ ]:
# python train_mlp.py --lang='en' --label_type1='pron' --label_type2='articulation' --dir_data='data/audio' --dir_list='/home/coldbrew/fluent/01.발음평가모델/1.모델소스코드/datasets_list' --mlp_hidden=64 --epochs=200 --patience=20 --batch_size=256 --dir_model='model_test1'

base wav2vec2 model: facebook/wav2vec2-large-robust-ft-libri-960h
Traceback (most recent call last):
  File "/home/coldbrew/fluent/01.발음평가모델/1.모델소스코드/train_mlp.py", line 235, in <module>
    train_mlp()
  File "/home/coldbrew/fluent/01.발음평가모델/1.모델소스코드/train_mlp.py", line 142, in train_mlp
    trn_feat_x, trn_feat_y = feat_extraction(args, 'trn') # feature extraction for training data
                             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/coldbrew/fluent/01.발음평가모델/1.모델소스코드/train_mlp.py", line 33, in feat_extraction
    filelist = open_file(fname_list)
               ^^^^^^^^^^^^^^^^^^^^^
  File "/home/coldbrew/fluent/01.발음평가모델/1.모델소스코드/train_mlp.py", line 23, in open_file
    f = open(filename)
        ^^^^^^^^^^^^^^
FileNotFoundError: [Errno 2] No such file or directory: '/mnt/f/fluent/037.교육용_한국인의_영어_음성_데이터/01-1.정식개방데이터/lang_en/pron_trn.list'


In [ ]:
python train_mlp.py --lang='en' --label_type1='pron' --label_type2='articulation' --dir_data='data/audio' --dir_list='/home/coldbrew/fluent/01.발음평가모델/1.모델소스코드/datasets_small_list' --mlp_hidden=64 --epochs=200 --patience=20 --batch_size=256 --dir_model='model_test1'

In [ ]:
/mnt/f/fluent/037.교육용_한국인의_영어_음성_데이터/01-1.정식개방데이터/


이러한 파일을 만드는 쉘스크립트 코드 만들어줘

*  pron_trn.list
*  pron_val.list
*  pron_test.list
이런 형식으로 3개의 텍스트 파일을 만드는데 
아래 폴더에 있는 모든 .wav파일을 가지고 파일이름을 리스트화하고 그게 맞는 점수는 
/mnt/f/fluent/037.교육용_한국인의_영어_음성_데이터/01-1.정식개방데이터/Training/01.원천데이터/TS_PRN_en_NA_NA/NA-NA-F-13-ko-220812-100000_133_1_3-en.wav

[articulation  score]는 json의 itemScript->rating->articulationScore
[prosody  score]는 json의 itemScript->rating->prosodyScore 에서 가져오면됨
/mnt/f/fluent/037.교육용_한국인의_영어_음성_데이터/01-1.정식개방데이터/Training/02.라벨링데이터/TL_PRN_en_NA_NA (2)/NA-NA-F-13-ko-220812-100000_133_1_3-en.json

필요한 리스트
*.list  :  [wav  file]  [articulation  score]  [prosody  score]  [script]


#!/bin/bash

# 설정할 디렉토리 경로
audio_dir="/mnt/f/fluent/037.교육용_한국인의_영어_음성_데이터/01-1.정식개방데이터/Training/01.원천데이터/TS_PRN_en_NA_NA"
label_dir="/mnt/f/fluent/037.교육용_한국인의_영어_음성_데이터/01-1.정식개방데이터/Training/02.라벨링데이터/TL_PRN_en_NA_NA"
output_dir="."

# 출력 파일 초기화
> "$output_dir/pron_trn.list"
> "$output_dir/pron_val.list"
> "$output_dir/pron_test.list"

# 트레인, 밸리데이션, 테스트 파일 리스트
train_list="$output_dir/pron_trn.list"
val_list="$output_dir/pron_val.list"
test_list="$output_dir/pron_test.list"

# 모든 .wav 파일에 대해 처리
find "$audio_dir" -name "*.wav" | while read -r wav_file; do
    # 대응하는 json 파일 경로 설정
    filename=$(basename "$wav_file" .wav)
    json_file="$label_dir/$filename.json"
    
    # json 파일이 존재하는지 확인
    if [ -f "$json_file" ]; then
        # 점수와 스크립트 추출
        articulation_score=$(jq -r '.itemScript.rating.articulationScore' "$json_file")
        prosody_score=$(jq -r '.itemScript.rating.prosodyScore' "$json_file")
        script=$(jq -r '.itemScript.script' "$json_file")
        
        # 결과 문자열 생성
        result="$wav_file\t$articulation_score\t$prosody_score\t$script"
        
        # 임의로 파일을 트레인, 밸리데이션, 테스트 셋에 분류
        hash_value=$(echo -n "$filename" | md5sum | cut -d' ' -f1)
        mod_value=$((0x${hash_value:0:2} % 10))
        
        if [ "$mod_value" -lt 7 ]; then
            echo -e "$result" >> "$train_list"
        elif [ "$mod_value" -lt 9 ]; then
            echo -e "$result" >> "$val_list"
        else
            echo -e "$result" >> "$test_list"
        fi
    else
        echo "Warning: $json_file not found."
    fi
done

echo "Files generated: pron_trn.list, pron_val.list, pron_test.list"
